In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
Data_folder = r'../../New/New/Analysis/'

In [4]:
OD_folder = os.path.join(Data_folder, "OD_Matrix")
Base_folder = os.path.join(Data_folder, "BaseAllocation")

In [59]:
Dictionary_folder = os.path.join(OD_folder, "ID_Dictionary")
Tract_folder = os.path.join(OD_folder, "Tract")
Blockgroup_folder = os.path.join(OD_folder, "BlockGroup")
Base_T_folder = os.path.join(Base_folder, "T")
Base_BG_folder = os.path.join(Base_folder, "BG")

In [9]:
newTFolder = r"../Data/Tract"
newBGFolder = r"../Data/BG"

In [28]:
# Get tract list
tractList = []
tractListBC = []
tractListCC = []
T_BC_list = os.listdir(os.path.join(Tract_folder, "BC"))
T_CC_list = os.listdir(os.path.join(Tract_folder, "CC"))
for t in T_BC_list:
    tractListBC.append(t.split("_")[1].split(".")[0])
for t in T_CC_list:
    tractListCC.append(t.split("_")[1].split(".")[0])
for ID in tractListBC:
    # print(ID)
    if ID in tractListCC:
        tractList.append(ID)
# Get block group list
BGList = []
BGListBC = []
BGListCC = []
BG_BC_list = os.listdir(os.path.join(Blockgroup_folder, "BC"))
BG_CC_list = os.listdir(os.path.join(Blockgroup_folder, "CC"))
for t in BG_BC_list:
    BGListBC.append(t.split("_")[1].split(".")[0])
for t in BG_CC_list:
    BGListCC.append(t.split("_")[1].split(".")[0])
for ID in BGListBC:
    # print(ID)
    if ID in BGListCC:
        BGList.append(ID)

In [29]:
# create folders for tract and blockgroup
for name in tractList:
    newFolderPath = os.path.join(newTFolder, name)
    if not os.path.exists(newFolderPath):
        os.mkdir(newFolderPath)
for name in BGList:
    newFolderPath = os.path.join(newBGFolder, name)
    if not os.path.exists(newFolderPath):
        os.mkdir(newFolderPath)

In [43]:
def getNumpyMatrix(file, custNum, baseNum, matrixType):
    df = pd.read_csv(file)
    OID = df['OriginID'].values.tolist()
    DID = df['DestinationID'].values.tolist()
    Length = df["Total_Length"].values.tolist()

    if matrixType == "CC":
        matrix = np.ones((custNum, custNum)) * 1000000000

    else:
        matrix = np.ones((baseNum, custNum)) * 1000000000

    for i in range(len(OID)):
        matrix[OID[i]-1][DID[i]-1] = Length[i]
    return matrix

In [58]:
for t in tractList:
    new_tract_folder = os.path.join(newTFolder, t)
    bcFile = os.path.join(Tract_folder, "BC", "BCT_" + t + ".csv")
    ccFile = os.path.join(Tract_folder, "CC", "CCT_" + t + ".csv")
    baseDic = os.path.join(OD_folder, "ID_Dictionary", "Tract", t, "base.csv")
    custDic = os.path.join(OD_folder, "ID_Dictionary", "Tract", t, "customer.csv")

    # get dictionary information and save to new place
    bDic_df = pd.read_csv(baseDic)
    baseNum = bDic_df.shape[0]
    bDic_df = bDic_df[['BID']]
    bDic_df.to_csv(os.path.join(new_tract_folder, "baseDic.csv"))
    # break
    cDic_df = pd.read_csv(custDic)
    custNum = cDic_df.shape[0]
    cDic_df = cDic_df[["CID"]]
    cDic_df.to_csv(os.path.join(new_tract_folder, "custDic.csv"))

    # get bc and cc matrix
    bcM = getNumpyMatrix(bcFile, custNum, baseNum, "BC")
    ccM = getNumpyMatrix(ccFile, custNum, baseNum, "CC")

    np.save(os.path.join(new_tract_folder, "bcM.npy"), bcM)
    np.save(os.path.join(new_tract_folder, "ccM.npy"), ccM)

    # get base locations
    for i in range(1, 6):
        locationInfo = pd.read_csv(os.path.join(Base_T_folder, str(i) + '.csv'))
        locationInfo = locationInfo[locationInfo["unitID"] == int(t)]
        locationInfo.to_csv(os.path.join(new_tract_folder, "Base_" + str(i) + ".csv"))
    


In [61]:
for BG in BGList:
    new_bg_folder = os.path.join(newBGFolder, BG)
    bcFile = os.path.join(Blockgroup_folder, "BC", "BCb_" + t + ".csv")
    ccFile = os.path.join(Blockgroup_folder, "CC", "CCb_" + t + ".csv")
    baseDic = os.path.join(OD_folder, "ID_Dictionary", "Block", t, "base.csv")
    custDic = os.path.join(OD_folder, "ID_Dictionary", "Block", t, "customer.csv")

    # get dictionary information and save to new place
    bDic_df = pd.read_csv(baseDic)
    baseNum = bDic_df.shape[0]
    bDic_df = bDic_df[['BID']]
    bDic_df.to_csv(os.path.join(new_bg_folder, "baseDic.csv"))
    # break
    cDic_df = pd.read_csv(custDic)
    custNum = cDic_df.shape[0]
    cDic_df = cDic_df[["CID"]]
    cDic_df.to_csv(os.path.join(new_bg_folder, "custDic.csv"))

    # get bc and cc matrix
    bcM = getNumpyMatrix(bcFile, custNum, baseNum, "BC")
    ccM = getNumpyMatrix(ccFile, custNum, baseNum, "CC")

    np.save(os.path.join(new_bg_folder, "bcM.npy"), bcM)
    np.save(os.path.join(new_bg_folder, "ccM.npy"), ccM)

    # get base locations
    for i in range(1, 6):
        locationInfo = pd.read_csv(os.path.join(Base_BG_folder, str(i) + '.csv'))
        locationInfo = locationInfo[locationInfo["unitID"] == int(t)]
        locationInfo.to_csv(os.path.join(new_bg_folder, "Base_" + str(i) + ".csv"))